In [1]:
import requests

url_search = 'https://aspetos.com/de/verzeichnis/bestatter?page='
page_max = 30
resp = requests.get(url_search)

In [2]:
import shutil
from os import makedirs

aspetos_local = 'aspetos/aspetos_page/'

makedirs(aspetos_local, exist_ok = True)
shutil.rmtree(aspetos_local)
makedirs(aspetos_local, exist_ok = True)

In [3]:
with open('1.html', 'w') as wf:
    wf.write(resp.text)

    for pagenum in range(1, page_max):
        url = url_search + str(pagenum)
#        print('Downloading :', url)
        resp = requests.get(url)

        fname = aspetos_local + str(pagenum) + '.html'
#        print('Saving: ', fname)
        with open(fname, 'w') as wf:
            wf.write(resp.text)
            
print('\n')
print('done...')



done...


In [4]:
from bs4 import BeautifulSoup

import re

regex = re.compile(r'[\n\r\t]')

import pandas as pd
import glob
import os.path

df_aspetos = pd.DataFrame()

for file_name in glob.glob(os.path.join(aspetos_local, '*.html')):
    with open(file_name) as html_file:
        soup = BeautifulSoup(html_file)
#        print (file_name)
        
        for i in soup.find_all('div', {'isotope-list mortician-list clearfix'}):
            company_all = soup.find_all('div')
            
            for select in company_all:
                company = getattr(select.find('h3'), 'text', None)
#                print (company)

                address = getattr(select.find('div', {'content-col col-lg-9 col-md-12 col-xs-9'}), 'text', None)
#                print (address)

                df_aspetos = df_aspetos.append(pd.DataFrame([[company, address]],
                                                            columns = ['Marke', 'Adresse']))

print('\n')
print('done...')



done...


In [5]:
df_aspetos.shape

(8292, 2)

In [6]:
df_aspetos.reset_index(drop = True, inplace = True)
#df_aspetos.head(25)

In [7]:
df_aspetos = df_aspetos[df_aspetos['Marke'].notna()]
df_aspetos = df_aspetos.drop_duplicates('Marke', keep = 'first')
#df_aspetos.head(25)

In [8]:
df_aspetos[['Strasse', 'Plz', 'Ort']] = df_aspetos.Adresse.str.extract('(.+)[ ]([0-9]{5})[ ](.+)', expand = True)
df_aspetos[['Strasse_4', 'Plz_4', 'Ort_4']] = df_aspetos.Adresse.str.extract('(.+)[ ]([0-9]{4})[ ](.+)', expand = True)
#df_aspetos.head(25)

In [9]:
df_aspetos.loc[df_aspetos['Strasse'].isnull(), 'Strasse'] = df_aspetos['Strasse_4']
df_aspetos.loc[df_aspetos['Plz'].isnull(), 'Plz'] = df_aspetos['Plz_4']
df_aspetos.loc[df_aspetos['Ort'].isnull(), 'Ort'] = df_aspetos['Ort_4']
#df_aspetos.head(25)

In [10]:
df_aspetos['Plz'] = df_aspetos['Plz'].astype(int)
df_aspetos['Plz'] = ["%05d" % n for n in df_aspetos['Plz']]
#df_aspetos.head(25)

In [11]:
df_aspetos['Source'] = 'https://aspetos.com/de/verzeichnis/bestatter'
#df_aspetos.head(25)

In [12]:
df_aspetos = df_aspetos.drop('Adresse', axis = 1)
df_aspetos = df_aspetos.drop('Strasse_4', axis = 1)
df_aspetos = df_aspetos.drop('Plz_4', axis = 1)
df_aspetos = df_aspetos.drop('Ort_4', axis = 1)
#df_aspetos.head(25)

In [13]:
df_aspetos

,Marke,Strasse,Plz,Ort,Source
13,Leffers Bestattungen,Heidelberg...,64625,Bensheim,https://aspetos.com/de/verzeichnis/bestatter
46,Lichtblicke Bestattungen,Tennenbach...,79106,Freiburg im Breisgau,https://aspetos.com/de/verzeichnis/bestatter
55,LK-Bestattungen- und Friedhofsdienste,Landsberge...,10249,Berlin,https://aspetos.com/de/verzeichnis/bestatter
64,LUNERA GmbH - Die Bestatter,Vaihinger ...,71063,Sindelfingen,https://aspetos.com/de/verzeichnis/bestatter
73,Maric Bestattungen,Seegefelde...,13583,Berlin,https://aspetos.com/de/verzeichnis/bestatter
...,...,...,...,...,...
8185,Hanrieder Bestattungen München-West,Sauerbruch...,81377,München,https://aspetos.com/de/verzeichnis/bestatter
8194,Hanrieder Bestattungen Unterschleißheim,Lindenstra...,85716,München,https://aspetos.com/de/verzeichnis/bestatter
8203,Hansa Bestattungen,Angerstraß...,22087,Hamburg,https://aspetos.com/de/verzeichnis/bestatter
8212,Hartmut Woite Bestattungen,Urbanstraß...,10961,Berlin Kreuzberg,https://aspetos.com/de/verzeichnis/bestatter


In [14]:
df_aspetos.to_csv('aspetos/df_aspetos_20200605.csv', sep = ';', index = False)

In [15]:
import pandas as pd

df = pd.read_csv('aspetos/df_aspetos_20200605.csv', sep = ';')
df

,Marke,Strasse,Plz,Ort,Source
0,Leffers Bestattungen,Heidelberg...,64625,Bensheim,https://aspetos.com/de/verzeichnis/bestatter
1,Lichtblicke Bestattungen,Tennenbach...,79106,Freiburg im Breisgau,https://aspetos.com/de/verzeichnis/bestatter
2,LK-Bestattungen- und Friedhofsdienste,Landsberge...,10249,Berlin,https://aspetos.com/de/verzeichnis/bestatter
3,LUNERA GmbH - Die Bestatter,Vaihinger ...,71063,Sindelfingen,https://aspetos.com/de/verzeichnis/bestatter
4,Maric Bestattungen,Seegefelde...,13583,Berlin,https://aspetos.com/de/verzeichnis/bestatter
...,...,...,...,...,...
535,Hanrieder Bestattungen München-West,Sauerbruch...,81377,München,https://aspetos.com/de/verzeichnis/bestatter
536,Hanrieder Bestattungen Unterschleißheim,Lindenstra...,85716,München,https://aspetos.com/de/verzeichnis/bestatter
537,Hansa Bestattungen,Angerstraß...,22087,Hamburg,https://aspetos.com/de/verzeichnis/bestatter
538,Hartmut Woite Bestattungen,Urbanstraß...,10961,Berlin Kreuzberg,https://aspetos.com/de/verzeichnis/bestatter
